# 目的  
家賃、駅徒歩などの条件に合う物件を見つけたい。作成したモデルを使ってお得物件を発見したい。  
学生時代新宿周辺に住んでいたこともあって、新宿区の物件に興味があるので調査  
### GCPを用いて、バッチ処理、スクレイピング、ストレージの一連の流れを実装

ライブラリインポート

In [2]:
from bs4 import BeautifulSoup
import urllib3
import re
import requests
import time
import pandas as pd
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
from sklearn.preprocessing import OrdinalEncoder
from sklearn import preprocessing
#探索的データ解析
import pandas_profiling as pdp 
pd.set_option('display.max_columns',50)
pd.set_option('display.max_rows',100)
import warnings
warnings.simplefilter('ignore')

%matplotlib inline
import matplotlib.pyplot as plt
import japanize_matplotlib
import lightgbm as lgb
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from google.cloud import storage as gcs # gcsへデータを送るのに必要
from google.cloud import bigquery as gbq # BigQueryのテーブルにデータを挿入するのに必要
import io
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import LabelEncoder


In [2]:
#滋賀県の5件のデータ
#https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=10&sc=10380&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2=

In [1]:
import requests
#Webスクレイピングによく使用されるライブラリの一つで、HTMLやXMLなどのマークアップ言語で書かれた文書をパースして、文書内の情報を抽出するための機能
from bs4 import BeautifulSoup
import pandas as pd
import time
import re
import datetime
from google.cloud import storage as gcs # gcsへデータを送るのに必要
from google.cloud import bigquery as gbq # BigQueryのテーブルにデータを挿入するのに必要
# スクレイピングの開始点のURL
url = "https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&ra=013&rn=0045&ek=004506820&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09&po2=99&pc=100"

#"https://suumo.jp/jj/chintai/ichiran/FR301FC001/?ar=030&bs=040&ta=10&sc=10380&cb=0.0&ct=9999999&et=9999999&cn=9999999&mb=0&mt=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&fw2="

# スクレイピングを実行する関数
def suumo_scraping():
    name = []
    station = []
    price = []
    sikikinreikin = []
    room = []
    age = []
    address = []
    count_new_list = [] 

    urls=requests.get(url)
    # 連続してアクセスするのを防ぐために3秒待つ
    time.sleep(3)
    #urls.encodingは、サイトの文字コードを表す文字列を返す。
     #urls.apparent_encodingを使って、文字コードが正しく設定されていない場合に、自動的に正しい文字コードを設定するために使用
    urls.encoding = urls.apparent_encoding 
    
    #、指定したURLから取得したHTMLドキュメントを解析し、特定の要素を抽出する
    soup=BeautifulSoup()
     #BeautifulSoupを使って、URLから取得したHTMLコンテンツを解析して、パース可能なオブジェクトを作成
     #第一引数には、解析したいHTMLのコンテンツが含まれた文字列を渡し、第二引数には、使用する解析器を指定します。ここでは、"html.parser"
    soup=BeautifulSoup(urls.content,"html.parser")
    #soupオブジェクトから、"ol"タグでclass属性が"pagination-parts"である要素を抽出
    get_url = soup.find("ol",class_="pagination-parts")
    
    # 物件のページ数を取得
    # 指定したHTMLドキュメントから"li"タグのテキストを抽出し、リストに格納
    num_list =[]
    for i in get_url.find_all("li"):  #get_urlというオブジェクトから、"li"タグのすべての要素を検索
        num_list.append(i.text)       #append()メソッドは、リストの最後に指定された値を追加するために使用されます。
                                      #ここでは、i.textをnum_listに追加しています。i.textは、i要素内に含まれるテキストを表している。
    num = int(num_list[10]) + 1       #リストnum_listの11番目の要素を取得し、整数値に変換してから1を足している。これにより、取得したページ数に1を加えた値がnumに格納される。

    
    # ページ数分だけスクレイピングを実行
    for p in range(1,num):
        page=str(p)
        url_2="https://suumo.jp/jj/chintai/ichiran/FR301FC005/?ar=030&bs=040&ra=013&rn=0045&ek=004506820&cb=0.0&ct=9999999&mb=0&mt=9999999&et=9999999&cn=9999999&shkr1=03&shkr2=03&shkr3=03&shkr4=03&sngz=&po1=09&po2=99&pc=100" + "&page=" + page
        urls=requests.get(url_2)  #requests.get()関数を使用してHTMLデータを取得
        # 連続してアクセスするのを防ぐために3秒待つ
        time.sleep(3)
        soup=BeautifulSoup(urls.content,"html.parser")
        
        # 「本日の新着」「新着」のタグを検索
         #soupのHTML内から「class="ellipse_pct ellipse_pct--red"」というクラスを持つ「<span>」タグを探し,
         #この「<span>」タグには、「本日の新着」という文言が含まれており、それを表すために使われています。このコードが見つけたすべての「<span>」タグは、リストとして変数「house_info」に格納
        house_info = soup.find_all("span",class_="ellipse_pct ellipse_pct--red")
        
        # 「本日の新着」「新着」のタグの中で「本日の新着」の数をカウント
        
         #各ページにおいて「本日の新着」の件数を数えて、リストとして保存
        count_new = 0
        for i in house_info:  #前のコードで取得した「<span>」タグのリスト「house_info」をループ処理
            if i.text == "本日の新着":  #その中に含まれるテキストが「本日の新着」である場合に、変数「count_new」を1つ増やす。
                count_new += 1
        count_new_list.append(count_new)  #各ページにおいて「本日の新着」が何件あるかをカウントし、「count_new_list」リストに追加
        
        # ページ内の「本日の新着」の数が0でないならスクレイピングを実行
        if count_new != 0:   
            #'<a>'タグの中から、classが"js-cassetLinkHref"であるものを所得
            house_name = soup.find_all("a",class_="js-cassetLinkHref")
            #最寄り駅の名前の取得
             #<div>タグの中から、styleがfont-weight:boldであるものをすべて取得
            station_name = soup.find_all("div",style="font-weight:bold")
            #'urls'からテーブルをスクレイピング
            table_data = pd.read_html(urls.content)
            #スクレイピングした情報をリストに追加
             #house_nameとstation_nameは、それぞれ物件名と最寄り駅名が含まれるタグを取得
            for h in house_name:
                name.append(h.text)
            for s in station_name:
                station.append(s.text)
            #物件情報の取得
            for i in range(0,count_new):
                #table_dataには、物件の価格、間取り、築年数、住所、敷金・礼金などがテーブル形式で格納
                table = table_data[i]
                #最初のテーブルの最初の行の最初の列に格納された価格をpriceリストに追加
                price.append(table.iloc[0,0])
                room.append(table.iloc[0,2])
                age.append(table.iloc[0,3])
                address.append(table.iloc[0,4])
                sikikinreikin.append(table.iloc[0,1])
        # ページ内の「本日の新着」が0ならスクレイピングをやめる
        else:
            break
    #スクレイピングした情報をリストにまとめて返す
    return name, station, price, sikikinreikin, room, age, address, count_new_list

# 前処理を実行する関数 スクレイピングの際に余計な空白等がついてくるのでここで除去する
# このコードだけだとイメージしにくい。手元の環境でデータを見ながらのほうがイメージしやすい
 #preprocessingの定義。suumo_scraping関数によって、取得したデータが格納されている。
def preprocessing(name, station, price, sikikinreikin, room, age, address, count_new_list):
    station_line_list = []
    time_move_list = []
 #stationリストから「駅名」と「路線名と所要時間」を分離し、それぞれstation_line_listとtime_move_listに追加
    for i in station:
        #split()メソッドを用いて、各駅名を空白で分割し、その結果得られるリストの0番目を路線名
        station_line_list.append(i.split(" ")[0])
        #1番目を所要時間として、それぞれstation_line_listとtime_move_listに追加
        time_move_list.append(i.split(" ")[1])

    station_list = []
    line_list = []

    for i in station_line_list:
        #station_line_listリストには、駅名と路線名が"/"でつながった文字列が格納されているため、まずはsplit()関数を使って"/"で分割し、駅名と路線名のリストを作成
        #、駅名と路線名をそれぞれappend()関数でstation_listとline_listに格納
        station_list.append(i.split("/")[1])
        line_list.append(i.split("/")[0])

    move_list =[]
    time_list = []

    for i in time_move_list:
        #time_move_list の要素をループで取り出して、文字列中に "歩" が含まれる場合は move_list に徒歩を追加
        if "歩" in i:
            move_list.append("徒歩")
            #移動時間に含まれる非数字の文字を除去するため、re.sub(r"\D", "", i) を使って、time_list に数値だけを追加
            time_list.append(re.sub(r"\D", "", i))
        #それ以外の場合は "バス" を追加
        else:
            move_list.append("バス")
            time_list.append(re.sub(r"\D", "", i))

    df=pd.DataFrame()        
    
    #それぞれのリストに各列を追加する。
    df["station"] = pd.Series(station_list)
    df["line"] = pd.Series(line_list)
    df["move"] = pd.Series(move_list)
    df["time_to_station_min"] = pd.Series(time_list).astype(float)

  #家賃と管理費をそれぞれ抜き出し、価格リストと管理費リストを作成するための処理
    price_list = []
    admin_list = []

    for i in price: #変数priceには、スクレイピングした不動産情報から取得した文字列型の価格情報がリストとして格納されている
       #priceリストから各要素を取り出して、スペースで分割した後に家賃と管理費を抜き出して、price_listとadmin_listに追加
        price_list.append(i.split(" ")[0].replace("万円",""))
        admin_list.append(i.split(" ")[3].replace("円",""))

    admin_fee_list = []
   #物件の管理費用を管理費用のリストから取得
    for i in admin_list: #admin_list リストの要素を一つずつ取り出し、i という名前で参照
        #i が "-" であれば、管理費用が存在しないと判断し、admin_fee_list に "0" を追加
        if i == "-":
            admin_fee_list.append("0")
        #i が "-" 以外であれば、管理費用が存在し、その値をadmin_fee_list に追加
        else:
            admin_fee_list.append(i)

    df["price_10k"] = pd.Series(price_list).astype(float)
    df["admin_fee"] = pd.Series(admin_fee_list).astype(float)

    sikikin_pre = []
    reikin_pre = []
    deposit_pre = []
    sikibiki_pre = []
#i.split(" ")[0]で敷金の金額を、i.split(" ")[2]で礼金の金額を、
#i.split(" ")[4]で敷引きの金額を、i.split(" ")[6]で敷引き以外の償却費の金額を取り出す
    for i in sikikinreikin:
        sikikin_pre.append(i.split(" ")[0])
        reikin_pre.append(i.split(" ")[2])
        deposit_pre.append(i.split(" ")[4])
        sikibiki_pre.append(i.split(" ")[6])

    sikikin_list = []
    reikin_list = []
    deposit_list = []
    sikibiki_list = []

    for i in sikikin_pre:
        a = i.split("敷")[1]
       #万円で表されている場合は、万円の部分を除いて数値のみを残し、万円で表されていない場合は、0として扱い
        if "万円" in a:
            sikikin_list.append(a.replace("万円",""))
        else:
            sikikin_list.append(0)

    for i in reikin_pre:
        a = i.split('礼')[1]
        if "万円" in a:
            reikin_list.append(a.replace("万円",""))
        else:
            reikin_list.append(0)

    for i in deposit_pre: #保証金
        a = i.split('\xa0')[1]
        if "万円" in a:
            deposit_list.append(a.replace("万円",""))
        else:
            deposit_list.append(0)

    for i in sikibiki_pre: #敷引き
        a = i.split("\xa0")[1]
        if "万円" in a:
            sikibiki_list.append(a.replace("万円",""))
        elif "-" in a:
            sikibiki_list.append(0)
        else:
            sikibiki_list.append("実費")

    df["sikikin_10k"] = pd.Series(sikikin_list).astype(float)
    df["reikin_10k"] = pd.Series(reikin_list).astype(float)
    df["deposit_10k"] = pd.Series(deposit_list).astype(float)
    df["sikibiki_10k"] = pd.Series(sikibiki_list)

    room_list = []
    area_pre = []
    direction_pre = []

    for i in room:
        room_list.append(i.split(" ")[0])
        area_pre.append(i.split(" ")[2])
        direction_pre.append(i.split(" ")[4])

    area_list = []

    for i in area_pre:
        area_list.append(i.replace("m2",""))

    df["room"] = pd.Series(room_list)
    df["area_m2"] = pd.Series(area_list)
    df["direction"] = pd.Series(direction_pre)

    type_list = []
    age_pre = []

    for i in age:
        type_list.append(i.split(" ")[0])
        age_pre.append(i.split(" ")[2])

    age_list = []

    for i in age_pre:
        if i == "新築":
            age_list.append(0)
        else:
            age_list.append(re.sub(r"\D", "", i))

    df["type"] = pd.Series(type_list)
    df["age_year"] = pd.Series(age_list).astype(float)

    df["scraping_date"] = datetime.date.today()
    df["name"] = pd.Series(name)
    df["address"] = pd.Series(address)

    id_range = 0
    for i in count_new_list:
        id_range = i + id_range

   #現在の日付からID番号のリストを作成する
    id_list = []
    
    #datetime.date.today()を使って、現在の日付を取得
    to_day = str(datetime.date.today())
    #日付の文字列を作成するために、"-"を削除
    to_day = to_day.replace("-","")
    #id_range回繰り返しを行い、iを文字列に変換してID番号を生成
    for i in range(0,id_range):
        if i < 10:
            i = str(i)
            id_list.append(to_day + "000"  + i)            
        elif i >= 10 and i < 100:
            i = str(i)
            id_list.append(to_day + "00" + i)
        elif i >= 100 and i < 1000:
            i = str(i)
            id_list.append(to_day + "0" + i)            
        else:
            i = str(i)
            id_list.append(to_day + i) 

    df["scraping_id"] = pd.Series(id_list)
    
    #dfを作成
    df = df.reindex(columns=["scraping_id","scraping_date","name","price_10k","age_year","admin_fee","sikikin_10k","reikin_10k","deposit_10k","sikibiki_10k","line","station","move","time_to_station_min","room","area_m2","direction","type","address"])
    # 最後のページのスクレイピングでは「本日の新着」以外のデータも混ざっている
    # pd.read_html(urls.content)で取得した賃貸価格など
    # それ以外で取得した物件名などは「本日の新着」分のデータしか取得していないのでNULLの部分が出てくる
    # NULLが入っているデータは余分なデータなので削除する
    df.dropna(inplace=True)
    
    return df
# データをGCSへcsvファイルで保存する関数
def send_storage(df):
    #project_id と bucket_name で指定されたGCSのプロジェクトとバケットを利用
    project_id = "rapid-stage-380007" 
    bucket_name = "suumo_bucket01" 
    
   #suumo_bucket01 バケット内の suumo_data/test ディレクトリにファイルを保存するよう指定
    #gcs_path_1 で指定されたパスに _YYYYMMDD.csv のファイル名を付けて、データフレーム df をアップロード
    gcs_path_1 = "suumo_data/test" # バケットのファイルまでのパス
    
    # ファイル名でスクレイピングした日付がわかるようにする
    to_day = str(datetime.date.today())
    to_day = to_day.replace("-","")
    
    client = gcs.Client(project_id)
    bucket = client.get_bucket(bucket_name)
    
    #bucket.blob() メソッドを使用して、アップロード先のファイルを作成
    blob_gcs_1 = bucket.blob(gcs_path_1 + "_" + to_day + ".csv")
    
    #to_csv() メソッドで作成した CSV データを、blob.upload_from_string() メソッドを使用してアップロード
     #blob オブジェクトには、先ほど作成したアップロード先のファイルが格納
    blob_gcs_1.upload_from_string(data=df.to_csv(index=False))

# BigQueryのテーブルにDFデータをインサートする関数
 #この関数を呼び出すことで、指定したテーブルにDataFrameのデータが挿入される
def bigquery_insert(df):
    #gbq.Client()を呼び出して、BigQuery APIクライアントを初期化
    client = gbq.Client()
    
    #client.get_table()を使用して、挿入先のテーブルを取得
    #suumo01_tableというテーブルを取得
    table = client.get_table("rapid-stage-380007.suumo01.suumo01_table")
    
    #client.insert_rows()を使用して、DataFrameの値をテーブルに挿入
    client.insert_rows(table, df.values.tolist())

# Cloud Functionで実行する関数
# Cloud Functionsのエントリポイントであるmain関数を定義
def main(event, context):
    #main関数内で、suumo_scraping関数を呼び出して、スクレイピングしたデータを取得
    name, station, price, sikikinreikin, room, age, address, count_new_list = suumo_scraping()

    #取得したデータを前処理するためにpreprocessing関数を呼び出します。
     #前処理されたデータは、dfというDataFrameオブジェクトに格納
    df = preprocessing(name, station, price, sikikinreikin, room, age, address, count_new_list)
    
    #send_storage関数を呼び出して、前処理されたデータをGoogle Cloud Storageにアップロード
    send_storage(df)
    
    #bigquery_insert関数を呼び出して、前処理されたデータをBigQueryに挿入
    bigquery_insert(df)

In [ ]:
#requiremensts.txt
beautifulsoup4>=4.11.1  
requests>=2.28.1  
pandas>=1.5.3  
google-cloud-storage>=2.7.0  
google-cloud-bigquery>=3.6.0

In [5]:
with open('suumo_data_test_20230411.txt', 'r',encoding='utf-8') as df:
    df_content = df.read()

In [10]:
#df_content

In [3]:
# タブ区切りのテキストファイルをDataFrameに変換する
df = pd.read_csv('suumo_data_test_20230411.txt',delimiter=',',names=['scraping_id','scraping_date','name','price_10k','age_year',
                                                          'admin_fee','sikikin_10k','reikin_10k','deposit_10k','sikibiki_10k',
                                                          'line','station','move','time_to_station_min','room',
                                                          'area_m2','direction','type','address'])

In [8]:
df = df.drop(0)

In [14]:
# 提出ファイル
df.to_csv('suumo_sp_folder/suumo_sp01.csv',index=False)

In [21]:
df1 = pd.read_csv('suumo_sp_folder/suumo_sp01.csv')

In [5]:
with open('suumo_data_test_20230411.txt', 'r',encoding='utf-8') as df:
    df_content = df.read()

In [10]:
#df_content

In [7]:
# タブ区切りのテキストファイルをDataFrameに変換する
df = pd.read_csv('suumo_data_test_20230411.txt',delimiter=',',names=['scraping_id','scraping_date','name','price_10k','age_year',
                                                          'admin_fee','sikikin_10k','reikin_10k','deposit_10k','sikibiki_10k',
                                                          'line','station','move','time_to_station_min','room',
                                                          'area_m2','direction','type','address'])

In [5]:
#0行目は必要ないので除去
df = df.drop(0)

In [14]:
# 提出ファイル
df.to_csv('suumo_sp_folder/suumo_sp01.csv',index=False)

In [20]:
df1 = pd.read_csv('suumo_sp_folder/suumo_sp01.csv')

# 2つ目の関数  
スクレイピングして作成したDFからお得な家賃の物件を抽出する。  
GCFに下記のコードを入力する。  

In [27]:
#main.py
#エントリーポイント：mp_main
import base64

from google.cloud import storage
import pandas as pd
import io
from sklearn.linear_model import LinearRegression
import datetime
from sklearn.preprocessing import LabelEncoder


def read_csv_from_gcs(bucket_name, file_path):
    """Reads a CSV file from Google Cloud Storage and returns a Pandas DataFrame."""
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_path)
    data = blob.download_as_string()
    df = pd.read_csv(io.BytesIO(data),dtype=str)
    return df

def write_csv_to_gcs(df, bucket_name, file_path):
    """Writes a Pandas DataFrame to a CSV file in Google Cloud Storage."""
    client = storage.Client()
    bucket = client.get_bucket(bucket_name)
    blob = bucket.blob(file_path)
    blob.upload_from_string(df.to_csv(index=False), 'text/csv')

def mp_main(event, context):
    """Triggered from a message on a Cloud Pub/Sub topic.
    Args:
         event (dict): Event payload.
         context (google.cloud.functions.Context): Metadata for the event.
    """
    #pubsub_message = base64.b64decode(event['data']).decode('utf-8')
    #print(pubsub_message)
    
    df1 = read_csv_from_gcs('suumo_bucket01', 'suumo_data/test_20230411.csv')

    #カテゴリ変数のカラムを指定して、まとめて処理
    def encode_categorical(df,cols):
        for col in cols:
            le = LabelEncoder()
            df[col] = pd.Series(le.fit_transform(df[col]))
    
        return df
    df1 = encode_categorical(df1,cols=["scraping_date","name","line","station","move","room","direction","type","address"])

    X = df1.drop('price_10k', axis=1)
    y = df1['price_10k']

    model = LinearRegression()
    model.fit(X, y)

    df2 = df1.copy()
    df2['market_rent'] = model.predict(X)
    df2['price_10k'] = df2['price_10k'].astype(float)
    df2['market_rent'] = df2['market_rent'].astype(float)

    df2['rent_difference'] = df2['price_10k'] - df2['market_rent']

    df2_sorted = df2.sort_values('rent_difference', ascending=True)
    write_csv_to_gcs(df2_sorted, 'suumo_bucket01', 'suumo_mp_data/mp_file01.csv')

    #if 'data' in event:
    #    pubsub_message = base64.b64decode(event['data']).decode('utf-8')
    #else:
    #    pubsub_message = None

# データをGCSへcsvファイルで保存する関数
def send_storage(df2_sorted):
    project_id = "rapid-stage-380007" 
    bucket_name = "suumo_bucket01" 
    gcs_path_2 = "suumo_mp_data/mp_file01" # バケットのファイルまでのパス
    # ファイル名でスクレイピングした日付がわかるようにする
    to_day = str(datetime.date.today())
    to_day = to_day.replace("-","")
    
    client = gcs.Client(project_id)
    bucket = client.get_bucket(bucket_name)
    blob_gcs_2 = bucket.blob(gcs_path_2 + "_" + to_day + ".csv")

    blob_gcs_2.upload_from_string(data=df2_sorted.to_csv(index=False))




#requirements.txt  
pandas>=1.4.1
google-cloud-storage>=2.1.0

scikit-learn>=1.0.2
lightGBM>=3.3.3

In [37]:
#カテゴリ変数のカラムを指定して、まとめて処理
for col in cols:
    le = LabelEncoder()
    df1[col] = pd.Series(le.fit_transform(df1[col]))

df1 = encode_categorical(df1,cols=["scraping_date","name","line","station","move","room","direction","type","address"])

X = df1.drop('price_10k', axis=1)
y = df1['price_10k']

model = LinearRegression()
model.fit(X, y)

df2 = df1.copy()
df2['market_rent'] = model.predict(X)
df2['price_10k'] = df2['price_10k'].astype(float)
df2['market_rent'] = df2['market_rent'].astype(float)

df2['rent_difference'] = df2['price_10k'] - df2['market_rent']

df2_sorted = df2.sort_values('rent_difference', ascending=True)


In [38]:
df2_sorted[['price_10k','market_rent','rent_difference']]

,price_10k,market_rent,rent_difference
284,44.0,52.727688,-8.727688
239,39.0,47.514988,-8.514988
282,43.0,50.893050,-7.893050
238,39.0,46.617498,-7.617498
440,71.0,78.601456,-7.601456
...,...,...,...
578,113.0,101.890808,11.109192
579,113.0,101.620807,11.379193
585,118.0,104.618361,13.381639
584,117.0,102.908852,14.091148


In [39]:
df2_sorted

,scraping_id,scraping_date,name,price_10k,age_year,admin_fee,sikikin_10k,reikin_10k,deposit_10k,sikibiki_10k,line,station,move,time_to_station_min,room,area_m2,direction,type,address,market_rent,rent_difference
284,284,0,14,44.0,5,19,0,0,0,0,1,0,0,13,10,82,8,1,33,52.727688,-8.727688
239,239,0,210,39.0,7,9,34,33,0,0,1,0,0,11,3,118,8,1,27,47.514988,-8.514988
282,282,0,162,43.0,3,23,0,0,0,0,1,0,0,3,1,76,7,1,36,50.893050,-7.893050
238,238,0,260,39.0,7,9,34,33,0,0,0,0,0,15,1,118,8,1,27,46.617498,-7.617498
440,440,0,202,71.0,10,19,57,57,0,0,3,0,0,1,2,128,5,1,22,78.601456,-7.601456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
578,578,0,135,113.0,18,0,91,83,0,0,1,0,0,4,7,184,4,1,34,101.890808,11.109192
579,579,0,171,113.0,0,25,0,0,0,0,3,0,0,13,7,175,4,1,52,101.620807,11.379193
585,585,0,122,118.0,2,0,97,88,0,0,1,0,0,13,8,185,0,2,29,104.618361,13.381639
584,584,0,36,117.0,0,19,93,87,0,0,1,0,0,18,4,171,8,1,18,102.908852,14.091148


# GCPを利用した。AutoMLによる分析自動化

aiplatform.googleapis.com：Google Cloud Platform (GCP) 上で提供される、機械学習のためのプラットフォームサービスの1つで、AutoML や TensorFlow などの機械学習ツールを提供している。  
このサービスを使用することで、機械学習モデルのトレーニング、モデルのデプロイ、予測の実行などが簡単に行える。

#### 手順1  
Google Cloud Consoleにログインし、AutoML Tablesの画面に移動  
#### 手順2  
左側のメニューから「データセット」をクリックし、新しいデータセットを作成します。データセット名、説明、ターゲットカラムなどの詳細を設定  
今回は  
データセット名：「test_dataset_01」  
データタイプと目標の選択：表形式の「回帰/分類」
リージョン：us-central1(アイオワ)  
データセットを作成を押す。  
データソースを選択：「Cloud Storage から CSV ファイルを選択」
インポートファイルのパスを'参照'から行う。  
#### 手順3  
データのアップロードが完了したら、「モデルのトレーニング」をクリックして、AutoMLがモデルを自動的にトレーニングするように指示 
具体的には  
Dataset：test_dataset_01  
Objective：Regression  
最適化の目標：RMSE(外れ値があるとは考えにくいため。本来は候補の指標はそれぞれ試した方が良い。)
#### 手順4  


In [21]:
pip show scikit-learn


Name: scikit-learnNote: you may need to restart the kernel to use updated packages.

Version: 1.0.2
Summary: A set of python modules for machine learning and data mining
Home-page: http://scikit-learn.org
Author: 
Author-email: 
License: new BSD
Location: c:\programdata\anaconda3\lib\site-packages
Requires: joblib, numpy, scipy, threadpoolctl
Required-by: lightgbm, scikit-learn-intelex, xfeat


In [7]:
pip show lightGBM

Name: lightgbmNote: you may need to restart the kernel to use updated packages.

Version: 3.3.3
Summary: LightGBM Python Package
Home-page: https://github.com/microsoft/LightGBM
Author: 
Author-email: 
License: The MIT License (Microsoft)
Location: c:\users\t-miy\appdata\roaming\python\python39\site-packages
Requires: numpy, scikit-learn, scipy, wheel
Required-by: xfeat


In [ ]:
    def encode_categorical(df,cols):
        for col in cols:
            le = LabelEncoder()
            df[col] = pd.Series(le.fit_transform(df[col]))
    
        return df
    df1 = encode_categorical(df1,cols=["scraping_date","name","line","station","move","room","direction","type","address"])

    X = df1.drop('price_10k', axis=1)
    y = df1['price_10k']

    model = LinearRegression()
    model.fit(X, y)

    df2 = df1.copy()
    df2['market_rent'] = model.predict(X)
    df2['price_10k'] = df2['price_10k'].astype(float)
    df2['market_rent'] = df2['market_rent'].astype(float)

    df2['rent_difference'] = df2['price_10k'] - df2['market_rent']

    df2_sorted = df2.sort_values('rent_difference', ascending=True)
    write_csv_to_gcs(df2_sorted, 'suumo_bucket01', 'suumo_mp_data/mp_file01.csv')
